In [1]:
import imodels
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from imodels.tree.rf_plus.rf_plus.rf_plus_models import RandomForestPlusRegressor
from sklearn.linear_model import LinearRegression
from sklearn.metrics import roc_auc_score, f1_score, recall_score, precision_score, mean_squared_error, r2_score
from imodels.tree.rf_plus.feature_importance.rfplus_explainer import *
from sklearn.preprocessing import StandardScaler
import copy
import matplotlib.pyplot as plt
import openml


/accounts/projects/binyu/zhongyuan_liang/.local/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
X, y, _ = imodels.get_clean_dataset("diabetes_regr") #diabetes_regr
# X = np.delete(X, 4,1)
# dataset = openml.datasets.get_dataset(588)
# X, y, _, _ = dataset.get_data(target=dataset.default_target_attribute, dataset_format="array")

fetching diabetes from sklearn


In [4]:
from imodels.imodels.importance import RandomForestPlusRegressor

rf_plus_model = RandomForestPlusRegressor()
rf_plus_model.fit(X, y)
mdi_plus_scores = rf_plus_model.get_mdi_plus_scores(X, y)

ModuleNotFoundError: No module named 'imodels.imodels'

In [3]:
np.argsort(-1*np.array([5,2,1,4]))

array([0, 3, 1, 2])

In [5]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=1)

In [6]:
from sklearn.linear_model import Ridge
est = RandomForestRegressor(n_estimators=1, min_samples_leaf=5, bootstrap=True, max_features=0.33, random_state=42)
est.fit(X_train, y_train)
# rf_plus_base = RandomForestPlusRegressor(rf_model=est, prediction_model=LinearRegression())
# rf_plus_base.fit(X_train, y_train)
rf_plus_base_inbag = RandomForestPlusRegressor(rf_model=est, include_raw=False, fit_on="inbag", prediction_model=LinearRegression())
rf_plus_base_inbag.fit(X_train, y_train)

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 16 concurrent workers.


In [5]:
# CHECK how many features are used in the random forest
# Extract the unique features used in the tree
tree = est.estimators_[0]
features_used = set(tree.tree_.feature[tree.tree_.feature != -2])
print("Indices of features used in the tree:", features_used)
print("Number of unique features used in the tree:", len(features_used))

Indices of features used in the tree: {0, 1, 2, 3, 4, 5, 6, 7, 8, 9}
Number of unique features used in the tree: 10


In [6]:
explainer = shap.TreeExplainer(est)
rf_plus_mdi = RFPlusMDI(rf_plus_base_inbag, evaluate_on="inbag", mode="only_k")
shap_lfi = explainer.shap_values(X_train, check_additivity=False)
lmdi_lfi= rf_plus_mdi.explain_linear_partial(X=X_train, y=None)

#### Single Data Point

In [7]:
sample_index = 0
sample = X_train[sample_index].reshape(1, -1)

# Get the single tree
tree = est.estimators_[0]

# Find features used in the decision path
used_features = set()
node_indicator = tree.decision_path(sample)
feature_index = tree.tree_.feature

# Collect features used in the path for the sample
path_nodes = node_indicator.indices
for node in path_nodes:
    if feature_index[node] != -2:  # Ignore leaf nodes
        used_features.add(feature_index[node])

# Print the feature indices used in the decision path
print("Features used in the path for X_train[0]:", sorted(used_features))
print("Feature names used:", [f"feature_{i}" for i in sorted(used_features)])

Features used in the path for X_train[0]: [5, 7, 8, 9]
Feature names used: ['feature_5', 'feature_7', 'feature_8', 'feature_9']


In [8]:
lmdi_lfi[0]

array([-5.76596047e-16,  4.47089993e-17, -1.87620181e-15,  5.27833453e-17,
       -3.01439228e-16,  1.55571055e+01,  2.83016987e-17, -4.71930417e+01,
       -1.47863611e+01,  3.82245322e+01])

In [9]:
shap_lfi[0]

array([  1.84433176,   0.31993887, -12.22467027,  -0.67490505,
         3.92190952,  18.93418583,  -0.84935214, -18.80385425,
       -16.27386257,  15.60851322])

#### Verify whether positive beta*x leads to positive y_hat changes

In [10]:
neg_shap_rank = np.argsort(shap_lfi)
neg_lmdi_rank = np.argsort(lmdi_lfi)
pos_shap_rank = np.argsort(-shap_lfi)
pos_lmdi_rank = np.argsort(-lmdi_lfi)

In [11]:
train_mean = np.mean(X_train, axis=0)
#train_mean = np.zeros(X_train.shape[1])

In [12]:
y_pred_train = est.predict(X_train)

In [13]:
data_copy = X_train.copy()
indices = neg_shap_rank[:, 0]
sum = 0
for i in range(X_train.shape[0]):
    if shap_lfi[i, indices[i]] < 0:
        if lmdi_lfi[i, indices[i]] < 0:
            data_copy[i, indices[i]] = 9999999
        else:
            data_copy[i, indices[i]] = -1*9999999
        sum += 1
y_pred_train_shap_neg = est.predict(data_copy)
array = y_pred_train_shap_neg-y_pred_train
print("sum:", sum)
np.sum(array != 0)

sum: 296


216

In [14]:
data_copy = X_train.copy()
indices = pos_shap_rank[:, 0]
sum = 0
for i in range(X_train.shape[0]):
    if shap_lfi[i, indices[i]] > 0:
        if lmdi_lfi[i, indices[i]] < 0:
            data_copy[i, indices[i]] = 9999999
        else:
            data_copy[i, indices[i]] = -1*9999999
        sum += 1
y_pred_train_shap_pos = est.predict(data_copy)
array = y_pred_train-y_pred_train_shap_pos
print("sum:", sum)
np.sum(array != 0)

sum: 296


215

In [15]:
data_copy = X_train.copy()
indices = neg_lmdi_rank[:, 0]
sum = 0
for i in range(X_train.shape[0]):
    if lmdi_lfi[i, indices[i]] < 0:
        if lmdi_lfi[i, indices[i]] < 0:
            data_copy[i, indices[i]] = 9999999
        else:
            data_copy[i, indices[i]] = -1*9999999
        sum += 1
y_pred_train_lmdi_neg = est.predict(data_copy)
array = y_pred_train_lmdi_neg-y_pred_train
print(f"Sum: {sum}")
np.sum(array != 0)

Sum: 296


248

In [16]:
data_copy = X_train.copy()
indices = pos_lmdi_rank[:, 0]
sum = 0
for i in range(X_train.shape[0]):
    if lmdi_lfi[i, indices[i]] > 0:
        if lmdi_lfi[i, indices[i]] < 0:
            data_copy[i, indices[i]] = 9999999
        else:
            data_copy[i, indices[i]] = -1*9999999
        sum += 1
    else:
        print(i)
y_pred_train_lmdi_pos = est.predict(data_copy)
array = y_pred_train-y_pred_train_lmdi_pos
print(f"Sum: {sum}")
np.sum(array != 0)

Sum: 296


247

In [19]:
data_copy = X_train.copy()
indices = pos_shap_rank[:, 0]
sum = 0
for i in range(X_train.shape[0]):
    if shap_lfi[i, indices[i]] > 0:
        if lmdi_lfi[i, indices[i]] < 0:
            data_copy[i, indices[i]] = 9999999
        else:
            data_copy[i, indices[i]] = -1*9999999
        sum += 1
y_pred_train_shap_pos = est.predict(data_copy)
array = y_pred_train-y_pred_train_shap_pos
print("sum:", sum)
np.sum(array > 0)

sum: 296


189

In [18]:
data_copy = X_train.copy()
indices = neg_shap_rank[:, 0]
sum = 0
for i in range(X_train.shape[0]):
    if shap_lfi[i, indices[i]] < 0:
        if lmdi_lfi[i, indices[i]] < 0:
            data_copy[i, indices[i]] = 9999999
        else:
            data_copy[i, indices[i]] = -1*9999999
        sum += 1
y_pred_train_shap_neg = est.predict(data_copy)
array = y_pred_train_shap_neg-y_pred_train
print("sum:", sum)
np.sum(array > 0)

sum: 296


194

In [17]:
data_copy = X_train.copy()
indices = pos_lmdi_rank[:, 0]
sum = 0
for i in range(X_train.shape[0]):
    if lmdi_lfi[i, indices[i]] > 0:
        if lmdi_lfi[i, indices[i]] < 0:
            data_copy[i, indices[i]] = 9999999
        else:
            data_copy[i, indices[i]] = -1*9999999
        sum += 1
    else:
        print(i)
y_pred_train_lmdi_pos = est.predict(data_copy)
array = y_pred_train-y_pred_train_lmdi_pos
print(f"Sum: {sum}")
np.sum(array > 0)

Sum: 296


197

In [20]:
data_copy = X_train.copy()
indices = neg_lmdi_rank[:, 0]
sum = 0
for i in range(X_train.shape[0]):
    if lmdi_lfi[i, indices[i]] < 0:
        if lmdi_lfi[i, indices[i]] < 0:
            data_copy[i, indices[i]] = 9999999
        else:
            data_copy[i, indices[i]] = -1*9999999
        sum += 1
y_pred_train_lmdi_neg = est.predict(data_copy)
array = y_pred_train_lmdi_neg-y_pred_train
print(f"Sum: {sum}")
np.sum(array > 0)

Sum: 296


192

In [213]:
lmdi_lfi[9]

array([ 1.37097070e+01,  4.47089993e-17, -1.87620181e-15, -2.38095238e-02,
       -3.01439228e-16, -3.71682314e-17,  2.83016987e-17, -4.71930417e+01,
        1.76298920e+01,  1.37261715e-16])

In [190]:
lmdi_lfi[0]

array([-5.76596047e-16,  4.47089993e-17, -1.87620181e-15,  5.27833453e-17,
       -3.01439228e-16,  1.55571055e+01,  2.83016987e-17, -4.71930417e+01,
       -1.47863611e+01,  3.82245322e+01])

In [191]:
array

array([  0.        ,   0.        ,   0.        ,   0.        ,
       109.59615385,   0.        ,  54.23333333,   0.        ,
        74.53333333,   0.        ,   0.        ,   0.        ,
         0.        ,  10.7       ,   0.        ,   0.        ,
         0.        ,  74.53333333,  74.9       ,  21.33333333,
         0.        , 110.84615385,  43.85      ,   3.44871795,
         0.        ,   0.        ,   0.        ,  64.125     ,
         0.        ,   0.        ,  83.2       ,   0.        ,
         0.        ,   0.        ,  96.64615385,   0.        ,
        24.5       ,  83.94444444,   0.        ,   0.        ,
        74.53333333,   0.        ,   0.        ,  24.5       ,
         0.        ,   0.        ,   0.        , -29.79166667,
       244.4       ,   0.        , 195.61666667,  22.58333333,
         0.        ,  43.85      ,  66.11666667,   0.        ,
        43.85      ,   0.        ,  19.91111111,   0.        ,
        74.53333333,   0.        ,  32.86111111,  74.53

In [192]:
X_train.shape

(296, 10)

In [193]:
rf_plus_mdi = RFPlusMDI(rf_plus_base_inbag, evaluate_on="inbag", mode="only_k")

In [194]:
est.predict(X_train[:10])

array([144.84615385, 138.1       , 159.15384615, 138.1       ,
       268.75      , 159.15384615, 192.33333333, 153.5       ,
       126.33333333, 137.16666667])

In [195]:
rf_plus_base_inbag.predict(X_train[:10])

array([144.84615385, 138.1       , 159.15384615, 138.1       ,
       268.75      , 159.15384615, 192.33333333, 153.5       ,
       126.33333333, 137.16666667])

In [196]:
lfi1 = rf_plus_mdi.explain_linear_partial(X=X_train, y=y_train)
lfi1

only_k
[-5.25105485e-18  1.42528632e-17  1.50030138e-18  0.00000000e+00
  0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
  0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
  0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
  0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
  0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
  0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
  0.00000000e+00]
only_k
[ 0.00000000e+00  0.00000000e+00  0.00000000e+00 -4.50090415e-18
  0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
  0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
  0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
  0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
  0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
  0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
  0.00000000e+00]
only_k
[ 0.00000000e+00  0.00000000e+00 

/accounts/projects/binyu/zhongyuan_liang/local_MDI+/imodels/imodels/tree/rf_plus/feature_importance/rfplus_explainer.py:297: RuntimeWarning: Mean of empty slice
  local_feature_importances = np.nanmean(local_feature_importances,axis=-1)


array([[-5.76596047e-16,  4.47089993e-17, -1.87620181e-15, ...,
        -4.71930417e+01, -1.47863611e+01,  3.82245322e+01],
       [-5.76596047e-16,  4.47089993e-17, -3.37966546e+01, ...,
         2.95604767e+01, -1.60744077e+01,  1.37261715e-16],
       [-5.76596047e-16,  4.47089993e-17,  2.88386876e+01, ...,
         2.95604767e+01, -2.08976287e+01,  1.37261715e-16],
       ...,
       [-5.76596047e-16,  4.47089993e-17, -1.87620181e-15, ...,
        -4.71930417e+01, -1.47863611e+01, -2.07049550e+01],
       [-5.76596047e-16,  4.47089993e-17, -3.37966546e+01, ...,
         2.95604767e+01, -1.60744077e+01,  1.37261715e-16],
       [            nan,             nan,             nan, ...,
                    nan,             nan,             nan]])

In [197]:
lfi1 = rf_plus_mdi.explain_linear_partial(X=X_train, y=None)
lfi1

only_k
[-5.25105485e-18  1.42528632e-17  1.50030138e-18  0.00000000e+00
  0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
  0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
  0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
  0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
  0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
  0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
  0.00000000e+00]
only_k
[ 0.00000000e+00  0.00000000e+00  0.00000000e+00 -4.50090415e-18
  0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
  0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
  0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
  0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
  0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
  0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
  0.00000000e+00]
only_k
[ 0.00000000e+00  0.00000000e+00 

array([[-5.76596047e-16,  4.47089993e-17, -1.87620181e-15, ...,
        -4.71930417e+01, -1.47863611e+01,  3.82245322e+01],
       [-5.76596047e-16,  4.47089993e-17, -3.37966546e+01, ...,
         2.95604767e+01, -1.60744077e+01,  1.37261715e-16],
       [-5.76596047e-16,  4.47089993e-17,  2.88386876e+01, ...,
         2.95604767e+01, -2.08976287e+01,  1.37261715e-16],
       ...,
       [-5.76596047e-16,  4.47089993e-17, -1.87620181e-15, ...,
        -4.71930417e+01, -1.47863611e+01, -2.07049550e+01],
       [-5.76596047e-16,  4.47089993e-17, -3.37966546e+01, ...,
         2.95604767e+01, -1.60744077e+01,  1.37261715e-16],
       [ 1.37097070e+01,  4.47089993e-17, -1.87620181e-15, ...,
        -4.71930417e+01,  1.76298920e+01,  1.37261715e-16]])

In [198]:
import numpy as np

# Check how many rows in lfi1 have NaN values
nan_rows = lfi1[np.isnan(lfi1).any(axis=1)]


In [199]:
nan_rows

array([], shape=(0, 10), dtype=float64)

In [200]:
lfi1[0]

array([-5.76596047e-16,  4.47089993e-17, -1.87620181e-15,  5.27833453e-17,
       -3.01439228e-16,  1.55571055e+01,  2.83016987e-17, -4.71930417e+01,
       -1.47863611e+01,  3.82245322e+01])

In [201]:
explainer = shap.TreeExplainer(est)
explainer.shap_values(X_train, check_additivity=False)[0]

array([  1.84433176,   0.31993887, -12.22467027,  -0.67490505,
         3.92190952,  18.93418583,  -0.84935214, -18.80385425,
       -16.27386257,  15.60851322])

In [202]:
explainer.shap_values(X_train, check_additivity=False)[0]

array([  1.84433176,   0.31993887, -12.22467027,  -0.67490505,
         3.92190952,  18.93418583,  -0.84935214, -18.80385425,
       -16.27386257,  15.60851322])

In [203]:
temp = X_train[0]
temp

array([ 0.01628068, -0.04464164, -0.046085  , -0.00567042, -0.07587041,
       -0.06143838, -0.01394774, -0.03949338, -0.05140387,  0.01963284])

In [204]:
temp[5] = 100

In [205]:
est.predict(np.array([temp]))

array([69.83333333])

In [206]:
explainer.expected_value

array([153.04391892])

In [207]:
alo_mdi = AloRFPlusMDI(rf_plus_base, evaluate_on="all", mode="only_k")

NameError: name 'rf_plus_base' is not defined

In [ ]:
lfi2 = alo_mdi.explain_linear_partial(X=X_train, y=y_train)
lfi2

In [ ]:
lfi1[0].sum()

In [ ]:
lfi2 = rf_plus_mdi.explain_linear_partial(X=X_train, y=y_train)
lfi2

In [ ]:
# np.random.seed(42) 
# data = np.random.randn(1000, 10)
# n_groups = 2
# group_indicator = np.random.choice(n_groups, size=1000)
# y = np.zeros(1000)
# coefficients = np.random.randn(n_groups, data.shape[1])
# for group in range(n_groups):
#     group_mask = group_indicator == group
#     selected_features = data[group_mask]
#     y[group_mask] = np.dot(selected_features, coefficients[group])
# X = np.column_stack((data, group_indicator))

In [ ]:
# Standardize the data
scaler = StandardScaler()
X = scaler.fit_transform(X)

In [ ]:
pd.DataFrame(X).corr()

In [ ]:
# Plot y
plt.hist(y, bins=50)

In [ ]:
# apply the log transformation to y
# y = np.log(y)
# plt.hist(y, bins=50)

In [ ]:
# X, y, _ = imodels.get_clean_dataset("diabetes_regr")
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=1)
print(X_train.shape, X_test.shape)
# standardize the data using sklearn's StandardScaler
# scaler = StandardScaler()
# X_train = scaler.fit_transform(X_train)
# X_test = scaler.transform(X_test)

In [ ]:
# est = RandomForestRegressor(n_estimators=100, min_samples_leaf=5, max_features=0.33, random_state=42)
# est.fit(X_train, y_train)
# rf_plus_base = RandomForestPlusRegressor(rf_model=est)
# rf_plus_base.fit(X_train, y_train)

In [ ]:
est = RandomForestClassifier(n_estimators=100, min_samples_leaf=3, max_features='sqrt', random_state=42)
est.fit(X_train, y_train)
rf_plus_base = RandomForestPlusClassifier(rf_model=est)
rf_plus_base.fit(X_train, y_train)

In [ ]:
import lime
import sys
sys.path.append('..')
sys.path.append('../..')
sys.path.append('.')
sys.path.append('./scripts')
from competing_methods_local import *

In [ ]:
a,b,c,d = LFI_evaluation_RFPlus_oob(X_train, y_train, X_train, y_train, X_test, y_test, X_test, y_test, fit=rf_plus_base, mode="absolute", train_only=False)

In [ ]:
lime_evaluation_RF(X_train, y_train, X_train, y_train, X_test, y_test, X_test, y_test, fit=est, mode="absolute", train_only=True)

In [ ]:
lime = lime_evaluation_RF(X_train, y_train, None, None, X_test, y_test, None, None, fit=est, mode="absolute", train_only=True)[0]

In [ ]:
treeshap = tree_shap_evaluation_RF(X_train, y_train, X_train, y_train, X_test, y_test, X_test, y_test, fit=est, mode="absolute", train_only=True)[0]

In [ ]:
lmdi = LFI_evaluation_RFPlus_all(X_train, y_train, X_train, y_train, X_test, y_test, X_test, y_test, fit=rf_plus_base, mode="absolute", train_only=True)[0]

In [ ]:
treeshap.shape

In [ ]:
column_means = np.mean(treeshap, axis=0)
sorted = np.argsort(-column_means)
sorted

In [ ]:
def select_top_features(array, sorted_indices, percentage):
    num_features = array.shape[1]
    num_selected = int(np.ceil(num_features * percentage))
    selected_indices = sorted_indices[:num_selected]
    selected_array = array[:, selected_indices]
    return num_selected, selected_array

In [ ]:
X_train[0]

In [ ]:
X_test[0]

In [ ]:
select_top_features(X_test, sorted, 0.25)[0]

In [ ]:
select_top_features(X_train, sorted, 0.25)[0]

In [ ]:
column_means = np.mean(lime, axis=0)
np.argsort(-column_means)

In [ ]:
column_means = np.mean(lmdi, axis=0)
np.argsort(-column_means)

In [ ]:
explainer = lime.lime_tabular.LimeTabularExplainer(X_train,verbose=False,mode="regression")
result = np.zeros((X_train.shape[0], X_train.shape[1]))
for i in range(X_train.shape[0]):
    exp = explainer.explain_instance(X_train[i,:], est.predict, num_features=X_train.shape[1])
    original_feature_importance = exp.as_map()[1]
    sorted_feature_importance = sorted(original_feature_importance,key = lambda x: x[0])
    for j in range(X_train.shape[1]):
        result[i,j] = sorted_feature_importance[j][1] #abs(sorted_feature_importance[j][1])

# Convert the array to a DataFrame
lime_values = pd.DataFrame(result, columns=[f'Feature_{i}' for i in range(X_train.shape[1])])
lime_values = lime_values

In [ ]:
lime_values

In [ ]:
explainer = lime.lime_tabular.LimeTabularExplainer(X_train,verbose=False,mode="regression")
result = np.zeros((X_train.shape[0], X_train.shape[1]))
i = 0
exp = explainer.explain_instance(X_train[i,:], est.predict, num_features=X_train.shape[1])

In [ ]:
exp.as_list()

In [ ]:
exp.as_map()[1]

In [ ]:
sorted_feature_importance = sorted(exp.as_map()[1],key = lambda x: x[0])
sorted_feature_importance

In [ ]:
sorted_feature_importance[0][1]

In [ ]:
def explain(self, X_train,X_test,num_features = 10): #For experiments change based on number of features we are ablating 
        
        # get shape of X_test
        if X_test is None: #assume we are explaining training set
            X_to_explain = copy.deepcopy(X_train)   
            n_samples, num_features = X_train.shape
        else: #assume we are explaining test set
            X_to_explain = copy.deepcopy(X_test)
            n_samples, num_features = X_test.shape
        
        # create data structure to save scores in
        result = np.zeros((n_samples, num_features))
        
        # initialize the LIME explainer
        explainer = lime.lime_tabular.LimeTabularExplainer(X_train,verbose=False,mode=self.task)
        
        for i in range(n_samples):
            exp = explainer.explain_instance(X_to_explain[i,:], self.model_pred_func,num_features=num_features)
            original_feature_importance = exp.as_map()[1]
            sorted_feature_importance = sorted(original_feature_importance,key = lambda x: x[0])
            for j in range(num_features):
                result[i,j] = sorted_feature_importance[j][1] #abs(sorted_feature_importance[j][1])
        
        # Convert the array to a DataFrame
        lime_values = pd.DataFrame(result, columns=[f'Feature_{i}' for i in range(num_features)])
        lime_values = lime_values #abs(lime_values)
        
        return lime_values

In [ ]:
from sklearn.kernel_ridge import KernelRidge
est = KernelRidge()
est.fit(X_train, y_train)
y_pred = est.predict(X_test)
print(mean_squared_error(y_test, y_pred))
print(r2_score(y_test, y_pred))

In [ ]:
from sklearn.model_selection import GridSearchCV
param_grid = {
    'alpha': [0.1, 1, 10, 100],
    'kernel': ['linear', 'poly', 'rbf', 'sigmoid'],
    'gamma': [0.1, 0.01, 0.001, None]
}
grid_search = GridSearchCV(KernelRidge(), param_grid, cv=5)
grid_search.fit(X_train, y_train)
best_est = grid_search.best_estimator_
y_pred = best_est.predict(X_test)
print(mean_squared_error(y_test, y_pred))
print(r2_score(y_test, y_pred))

In [ ]:
grid_search.best_params_

In [ ]:
rf = RandomForestClassifier(n_estimators=100, min_samples_leaf= 3, max_features= 'sqrt', random_state= 42)
rf.fit(X_train, y_train)
rf_plus_base = RandomForestPlusClassifier(rf_model=rf)
rf_plus_base.fit(X_train, y_train)

In [ ]:
rf_plus_mdi = AloRFPlusMDI(rf_plus_base, evaluate_on="all")
partial_preds_subtract_intercept = rf_plus_mdi.explain_subtract_intercept(X=X_test)

In [ ]:
partial_preds_subtract_intercept[0]

In [ ]:
partial_preds_subtract_intercept[0]

In [ ]:
rf_plus_base.predict_proba(X_test)

In [ ]:
temp = rf_plus_mdi.explain(X=X_test, y=y_test)

In [ ]:
temp[0][0]

In [ ]:
rf = RandomForestRegressor(n_estimators=100, random_state=41, min_samples_leaf=5, max_features=0.33)
rf.fit(X_train, y_train)
rf_plus_base = RandomForestPlusRegressor(rf_model=copy.deepcopy(rf))
rf_plus_base.fit(X_train, y_train)
test_all_mse_rf = mean_squared_error(y_test, rf.predict(X_test))
test_all_r2_rf = r2_score(y_test, rf.predict(X_test))
test_all_mse_rf_plus = mean_squared_error(y_test, rf_plus_base.predict(X_test))
test_all_r2_rf_plus = r2_score(y_test, rf_plus_base.predict(X_test))
print("Test MSE RF: ", test_all_mse_rf)
print("Test R2 RF: ", test_all_r2_rf)
print("Test MSE RF+: ", test_all_mse_rf_plus)
print("Test R2 RF+: ", test_all_r2_rf_plus)

In [ ]:
# Get shap values
import shap
explainer = shap.TreeExplainer(rf)
shap_values_train = explainer.shap_values(X_train, check_additivity=True)
# shap_values_train = np.abs(shap_values_train)
shap_values_test = explainer.shap_values(X_test, check_additivity=True)
# shap_values_test = np.abs(shap_values_test)

In [ ]:
def ablation_removal(train_mean, data, feature_importance_rank, feature_index):
    data_copy = data.copy()
    indices = feature_importance_rank[:, feature_index]
    data_copy[np.arange(data.shape[0]), indices] = train_mean[indices]
    return data_copy
def ablation_removal_positive(train_mean, data, feature_importance_rank, feature_importance, feature_index):
    data_copy = data.copy()
    indices = feature_importance_rank[:, feature_index]
    sum = 0
    for i in range(data.shape[0]):
        if feature_importance[i, indices[i]] > 0:
            sum += 1
            data_copy[i, indices[i]] = train_mean[indices[i]]
    print("Remove sum: ", sum)
    return data_copy
def ablation_removal_negative(train_mean, data, feature_importance_rank, feature_importance, feature_index):
    data_copy = data.copy()
    indices = feature_importance_rank[:, feature_index]
    sum = 0
    for i in range(data.shape[0]):
        if feature_importance[i, indices[i]] < 0:
            sum += 1
            data_copy[i, indices[i]] = train_mean[indices[i]]
    print("Remove sum: ", sum)
    return data_copy

In [ ]:
ablation_models = {"RF_Regressor": rf,
                    "Linear": LinearRegression(),
                    "RF_Plus_Regressor": rf_plus_base}
X_data = X_test
y_data = y_test
ablation_data="test"
ablation_models["Linear"].fit(X_train, y_train)

### Shap

In [ ]:
a_model = "Linear"
ablation_est = ablation_models[a_model]
metric_results = {}
local_fi_score_data =shap_values_test
local_fi_score_data_rank = np.argsort(shap_values_test)
num_ablate_features = X_train.shape[1]
train_mean = np.mean(X_train, axis=0)
y_pred = ablation_est.predict(X_data)
metric_results[a_model + f'_{ablation_data}_MSE_before_ablation'] = mean_squared_error(y_data, y_pred)
metric_results[a_model + f'_{ablation_data}_R_2_before_ablation'] = r2_score(y_data, y_pred)
ablation_results_list = [0] * num_ablate_features
ablation_results_list_r2 = [0] * num_ablate_features
X_temp = X_data.copy()
for i in range(num_ablate_features):
    print(f"enter i: {i}")
    ablation_X_data = ablation_removal_negative(train_mean, X_temp, local_fi_score_data_rank, local_fi_score_data, i)
    ablation_results_list[i] = mean_squared_error(y_data, ablation_est.predict(ablation_X_data))
    ablation_results_list_r2[i] = r2_score(y_data, ablation_est.predict(ablation_X_data))
    X_temp = ablation_X_data
for i in range(num_ablate_features):
    metric_results[f'{a_model}_{ablation_data}_MSE_after_ablation_{i+1}'] = ablation_results_list[i]
    metric_results[f'{a_model}_{ablation_data}_R_2_after_ablation_{i+1}'] = ablation_results_list_r2[i]

mse_before = metric_results[a_model + f'_{ablation_data}_MSE_before_ablation']
r2_before = metric_results[a_model + f'_{ablation_data}_R_2_before_ablation']

mse_after = [metric_results[f'{a_model}_{ablation_data}_MSE_after_ablation_{i}'] for i in range(1, num_ablate_features)]
r2_after = [metric_results[f'{a_model}_{ablation_data}_R_2_after_ablation_{i}'] for i in range(1, num_ablate_features)]

# Plotting MSE
plt.figure(figsize=(12, 6))
plt.plot(['Before'] + [f'After {i}' for i in range(1, num_ablate_features)], [mse_before] + mse_after, color='tab:red', marker='o')
plt.xlabel('Ablation Step')
plt.ylabel('MSE')
plt.title(f'{a_model} MSE Before and After Ablation')
plt.show()

# Plotting R^2
plt.figure(figsize=(12, 6))
plt.plot(['Before'] + [f'After {i}' for i in range(1, num_ablate_features)], [r2_before] + r2_after, color='tab:blue', marker='o')
plt.xlabel('Ablation Step')
plt.ylabel('R^2')
plt.title(f'{a_model} R^2 Before and After Ablation')
plt.show()

In [ ]:
a_model = "Linear"
ablation_est = ablation_models[a_model]
metric_results = {}
local_fi_score_data =shap_values_test
local_fi_score_data_rank = np.argsort(-1*shap_values_test)
num_ablate_features = X_train.shape[1]
train_mean = np.mean(X_train, axis=0)
y_pred = ablation_est.predict(X_data)
metric_results[a_model + f'_{ablation_data}_MSE_before_ablation'] = mean_squared_error(y_data, y_pred)
metric_results[a_model + f'_{ablation_data}_R_2_before_ablation'] = r2_score(y_data, y_pred)
ablation_results_list = [0] * num_ablate_features
ablation_results_list_r2 = [0] * num_ablate_features
X_temp = X_data.copy()
for i in range(num_ablate_features):
    ablation_X_data = ablation_removal_positive(train_mean, X_temp, local_fi_score_data_rank, local_fi_score_data, i)
    ablation_results_list[i] = mean_squared_error(y_data, ablation_est.predict(ablation_X_data))
    ablation_results_list_r2[i] = r2_score(y_data, ablation_est.predict(ablation_X_data))
    X_temp = ablation_X_data
for i in range(num_ablate_features):
    metric_results[f'{a_model}_{ablation_data}_MSE_after_ablation_{i+1}'] = ablation_results_list[i]
    metric_results[f'{a_model}_{ablation_data}_R_2_after_ablation_{i+1}'] = ablation_results_list_r2[i]

mse_before = metric_results[a_model + f'_{ablation_data}_MSE_before_ablation']
r2_before = metric_results[a_model + f'_{ablation_data}_R_2_before_ablation']

mse_after = [metric_results[f'{a_model}_{ablation_data}_MSE_after_ablation_{i}'] for i in range(1, num_ablate_features)]
r2_after = [metric_results[f'{a_model}_{ablation_data}_R_2_after_ablation_{i}'] for i in range(1, num_ablate_features)]

# Plotting MSE
plt.figure(figsize=(12, 6))
plt.plot(['Before'] + [f'After {i}' for i in range(1, num_ablate_features)], [mse_before] + mse_after, color='tab:red', marker='o')
plt.xlabel('Ablation Step')
plt.ylabel('MSE')
plt.title(f'{a_model} MSE Before and After Ablation')
plt.show()

# Plotting R^2
plt.figure(figsize=(12, 6))
plt.plot(['Before'] + [f'After {i}' for i in range(1, num_ablate_features)], [r2_before] + r2_after, color='tab:blue', marker='o')
plt.xlabel('Ablation Step')
plt.ylabel('R^2')
plt.title(f'{a_model} R^2 Before and After Ablation')
plt.show()

In [ ]:
a_model = "Linear"
ablation_est = ablation_models[a_model]
metric_results = {}
local_fi_score_data_rank = np.argsort(-1*np.abs(shap_values_test))#np.argsort(-1*local_feature_importances_train) #np.argsort(-1*shap_values_train) #np.argsort(-1*np.random.rand(X_train.shape[0], 10))
num_ablate_features = X_train.shape[1]
train_mean = np.mean(X_train, axis=0)

y_pred = ablation_est.predict(X_data)
metric_results[a_model + f'_{ablation_data}_MSE_before_ablation'] = mean_squared_error(y_data, y_pred)
metric_results[a_model + f'_{ablation_data}_R_2_before_ablation'] = r2_score(y_data, y_pred)
ablation_results_list = [0] * num_ablate_features
ablation_results_list_r2 = [0] * num_ablate_features
X_temp = X_data.copy()
for i in range(num_ablate_features):
    ablation_X_data = ablation_removal(train_mean, X_temp, local_fi_score_data_rank, i)
    ablation_results_list[i] = mean_squared_error(y_data, ablation_est.predict(ablation_X_data))
    ablation_results_list_r2[i] = r2_score(y_data, ablation_est.predict(ablation_X_data))
    X_temp = ablation_X_data
for i in range(num_ablate_features):
    metric_results[f'{a_model}_{ablation_data}_MSE_after_ablation_{i+1}'] = ablation_results_list[i]
    metric_results[f'{a_model}_{ablation_data}_R_2_after_ablation_{i+1}'] = ablation_results_list_r2[i]

mse_before = metric_results[a_model + f'_{ablation_data}_MSE_before_ablation']
r2_before = metric_results[a_model + f'_{ablation_data}_R_2_before_ablation']

mse_after = [metric_results[f'{a_model}_{ablation_data}_MSE_after_ablation_{i}'] for i in range(1, num_ablate_features)]
r2_after = [metric_results[f'{a_model}_{ablation_data}_R_2_after_ablation_{i}'] for i in range(1, num_ablate_features)]

# Plotting MSE
plt.figure(figsize=(12, 6))
plt.plot(['Before'] + [f'After {i}' for i in range(1, num_ablate_features)], [mse_before] + mse_after, color='tab:red', marker='o')
plt.xlabel('Ablation Step')
plt.ylabel('MSE')
plt.title(f'{a_model} MSE Before and After Ablation')
plt.show()

# Plotting R^2
plt.figure(figsize=(12, 6))
plt.plot(['Before'] + [f'After {i}' for i in range(1, num_ablate_features)], [r2_before] + r2_after, color='tab:blue', marker='o')
plt.xlabel('Ablation Step')
plt.ylabel('R^2')
plt.title(f'{a_model} R^2 Before and After Ablation')
plt.show()

In [ ]:
a_model = "RF_Regressor"
ablation_est = ablation_models[a_model]
metric_results = {}
local_fi_score_data =shap_values_test
local_fi_score_data_rank = np.argsort(shap_values_test)
num_ablate_features = X_train.shape[1]
train_mean = np.mean(X_train, axis=0)
y_pred = ablation_est.predict(X_data)
metric_results[a_model + f'_{ablation_data}_MSE_before_ablation'] = mean_squared_error(y_data, y_pred)
metric_results[a_model + f'_{ablation_data}_R_2_before_ablation'] = r2_score(y_data, y_pred)
ablation_results_list = [0] * num_ablate_features
ablation_results_list_r2 = [0] * num_ablate_features
X_temp = X_data.copy()
for i in range(num_ablate_features):
    print(f"enter i: {i}")
    ablation_X_data = ablation_removal_negative(train_mean, X_temp, local_fi_score_data_rank, local_fi_score_data, i)
    ablation_results_list[i] = mean_squared_error(y_data, ablation_est.predict(ablation_X_data))
    ablation_results_list_r2[i] = r2_score(y_data, ablation_est.predict(ablation_X_data))
    X_temp = ablation_X_data
for i in range(num_ablate_features):
    metric_results[f'{a_model}_{ablation_data}_MSE_after_ablation_{i+1}'] = ablation_results_list[i]
    metric_results[f'{a_model}_{ablation_data}_R_2_after_ablation_{i+1}'] = ablation_results_list_r2[i]

mse_before = metric_results[a_model + f'_{ablation_data}_MSE_before_ablation']
r2_before = metric_results[a_model + f'_{ablation_data}_R_2_before_ablation']

mse_after = [metric_results[f'{a_model}_{ablation_data}_MSE_after_ablation_{i}'] for i in range(1, num_ablate_features)]
r2_after = [metric_results[f'{a_model}_{ablation_data}_R_2_after_ablation_{i}'] for i in range(1, num_ablate_features)]

# Plotting MSE
plt.figure(figsize=(12, 6))
plt.plot(['Before'] + [f'After {i}' for i in range(1, num_ablate_features)], [mse_before] + mse_after, color='tab:red', marker='o')
plt.xlabel('Ablation Step')
plt.ylabel('MSE')
plt.title(f'{a_model} MSE Before and After Ablation')
plt.show()

# Plotting R^2
plt.figure(figsize=(12, 6))
plt.plot(['Before'] + [f'After {i}' for i in range(1, num_ablate_features)], [r2_before] + r2_after, color='tab:blue', marker='o')
plt.xlabel('Ablation Step')
plt.ylabel('R^2')
plt.title(f'{a_model} R^2 Before and After Ablation')
plt.show()

In [ ]:
a_model = "RF_Regressor"
ablation_est = ablation_models[a_model]
metric_results = {}
local_fi_score_data =shap_values_test
local_fi_score_data_rank = np.argsort(-1*shap_values_test)
num_ablate_features = X_train.shape[1]
train_mean = np.mean(X_train, axis=0)
y_pred = ablation_est.predict(X_data)
metric_results[a_model + f'_{ablation_data}_MSE_before_ablation'] = mean_squared_error(y_data, y_pred)
metric_results[a_model + f'_{ablation_data}_R_2_before_ablation'] = r2_score(y_data, y_pred)
ablation_results_list = [0] * num_ablate_features
ablation_results_list_r2 = [0] * num_ablate_features
X_temp = X_data.copy()
for i in range(num_ablate_features):
    ablation_X_data = ablation_removal_positive(train_mean, X_temp, local_fi_score_data_rank, local_fi_score_data, i)
    ablation_results_list[i] = mean_squared_error(y_data, ablation_est.predict(ablation_X_data))
    ablation_results_list_r2[i] = r2_score(y_data, ablation_est.predict(ablation_X_data))
    X_temp = ablation_X_data
for i in range(num_ablate_features):
    metric_results[f'{a_model}_{ablation_data}_MSE_after_ablation_{i+1}'] = ablation_results_list[i]
    metric_results[f'{a_model}_{ablation_data}_R_2_after_ablation_{i+1}'] = ablation_results_list_r2[i]

mse_before = metric_results[a_model + f'_{ablation_data}_MSE_before_ablation']
r2_before = metric_results[a_model + f'_{ablation_data}_R_2_before_ablation']

mse_after = [metric_results[f'{a_model}_{ablation_data}_MSE_after_ablation_{i}'] for i in range(1, num_ablate_features)]
r2_after = [metric_results[f'{a_model}_{ablation_data}_R_2_after_ablation_{i}'] for i in range(1, num_ablate_features)]

# Plotting MSE
plt.figure(figsize=(12, 6))
plt.plot(['Before'] + [f'After {i}' for i in range(1, num_ablate_features)], [mse_before] + mse_after, color='tab:red', marker='o')
plt.xlabel('Ablation Step')
plt.ylabel('MSE')
plt.title(f'{a_model} MSE Before and After Ablation')
plt.show()

# Plotting R^2
plt.figure(figsize=(12, 6))
plt.plot(['Before'] + [f'After {i}' for i in range(1, num_ablate_features)], [r2_before] + r2_after, color='tab:blue', marker='o')
plt.xlabel('Ablation Step')
plt.ylabel('R^2')
plt.title(f'{a_model} R^2 Before and After Ablation')
plt.show()

In [ ]:
a_model = "RF_Regressor"
ablation_est = ablation_models[a_model]
metric_results = {}
local_fi_score_data_rank = np.argsort(-1*np.abs(shap_values_test))#np.argsort(-1*local_feature_importances_train) #np.argsort(-1*shap_values_train) #np.argsort(-1*np.random.rand(X_train.shape[0], 10))
num_ablate_features = X_train.shape[1]
train_mean = np.mean(X_train, axis=0)

y_pred = ablation_est.predict(X_data)
metric_results[a_model + f'_{ablation_data}_MSE_before_ablation'] = mean_squared_error(y_data, y_pred)
metric_results[a_model + f'_{ablation_data}_R_2_before_ablation'] = r2_score(y_data, y_pred)
ablation_results_list = [0] * num_ablate_features
ablation_results_list_r2 = [0] * num_ablate_features
X_temp = X_data.copy()
for i in range(num_ablate_features):
    ablation_X_data = ablation_removal(train_mean, X_temp, local_fi_score_data_rank, i)
    ablation_results_list[i] = mean_squared_error(y_data, ablation_est.predict(ablation_X_data))
    ablation_results_list_r2[i] = r2_score(y_data, ablation_est.predict(ablation_X_data))
    X_temp = ablation_X_data
for i in range(num_ablate_features):
    metric_results[f'{a_model}_{ablation_data}_MSE_after_ablation_{i+1}'] = ablation_results_list[i]
    metric_results[f'{a_model}_{ablation_data}_R_2_after_ablation_{i+1}'] = ablation_results_list_r2[i]

mse_before = metric_results[a_model + f'_{ablation_data}_MSE_before_ablation']
r2_before = metric_results[a_model + f'_{ablation_data}_R_2_before_ablation']

mse_after = [metric_results[f'{a_model}_{ablation_data}_MSE_after_ablation_{i}'] for i in range(1, num_ablate_features)]
r2_after = [metric_results[f'{a_model}_{ablation_data}_R_2_after_ablation_{i}'] for i in range(1, num_ablate_features)]

# Plotting MSE
plt.figure(figsize=(12, 6))
plt.plot(['Before'] + [f'After {i}' for i in range(1, num_ablate_features)], [mse_before] + mse_after, color='tab:red', marker='o')
plt.xlabel('Ablation Step')
plt.ylabel('MSE')
plt.title(f'{a_model} MSE Before and After Ablation')
plt.show()

# Plotting R^2
plt.figure(figsize=(12, 6))
plt.plot(['Before'] + [f'After {i}' for i in range(1, num_ablate_features)], [r2_before] + r2_after, color='tab:blue', marker='o')
plt.xlabel('Ablation Step')
plt.ylabel('R^2')
plt.title(f'{a_model} R^2 Before and After Ablation')
plt.show()

In [ ]:
a_model = "RF_Plus_Regressor"
ablation_est = ablation_models[a_model]
metric_results = {}
local_fi_score_data =shap_values_test
local_fi_score_data_rank = np.argsort(shap_values_test)
num_ablate_features = X_train.shape[1]
train_mean = np.mean(X_train, axis=0)
y_pred = ablation_est.predict(X_data)
metric_results[a_model + f'_{ablation_data}_MSE_before_ablation'] = mean_squared_error(y_data, y_pred)
metric_results[a_model + f'_{ablation_data}_R_2_before_ablation'] = r2_score(y_data, y_pred)
ablation_results_list = [0] * num_ablate_features
ablation_results_list_r2 = [0] * num_ablate_features
X_temp = X_data.copy()
for i in range(num_ablate_features):
    print(f"enter i: {i}")
    ablation_X_data = ablation_removal_negative(train_mean, X_temp, local_fi_score_data_rank, local_fi_score_data, i)
    ablation_results_list[i] = mean_squared_error(y_data, ablation_est.predict(ablation_X_data))
    ablation_results_list_r2[i] = r2_score(y_data, ablation_est.predict(ablation_X_data))
    X_temp = ablation_X_data
for i in range(num_ablate_features):
    metric_results[f'{a_model}_{ablation_data}_MSE_after_ablation_{i+1}'] = ablation_results_list[i]
    metric_results[f'{a_model}_{ablation_data}_R_2_after_ablation_{i+1}'] = ablation_results_list_r2[i]

mse_before = metric_results[a_model + f'_{ablation_data}_MSE_before_ablation']
r2_before = metric_results[a_model + f'_{ablation_data}_R_2_before_ablation']

mse_after = [metric_results[f'{a_model}_{ablation_data}_MSE_after_ablation_{i}'] for i in range(1, num_ablate_features)]
r2_after = [metric_results[f'{a_model}_{ablation_data}_R_2_after_ablation_{i}'] for i in range(1, num_ablate_features)]

# Plotting MSE
plt.figure(figsize=(12, 6))
plt.plot(['Before'] + [f'After {i}' for i in range(1, num_ablate_features)], [mse_before] + mse_after, color='tab:red', marker='o')
plt.xlabel('Ablation Step')
plt.ylabel('MSE')
plt.title(f'{a_model} MSE Before and After Ablation')
plt.show()

# Plotting R^2
plt.figure(figsize=(12, 6))
plt.plot(['Before'] + [f'After {i}' for i in range(1, num_ablate_features)], [r2_before] + r2_after, color='tab:blue', marker='o')
plt.xlabel('Ablation Step')
plt.ylabel('R^2')
plt.title(f'{a_model} R^2 Before and After Ablation')
plt.show()

In [ ]:
a_model = "RF_Plus_Regressor"
ablation_est = ablation_models[a_model]
metric_results = {}
local_fi_score_data =shap_values_test
local_fi_score_data_rank = np.argsort(-1*shap_values_test)
num_ablate_features = X_train.shape[1]
train_mean = np.mean(X_train, axis=0)
y_pred = ablation_est.predict(X_data)
metric_results[a_model + f'_{ablation_data}_MSE_before_ablation'] = mean_squared_error(y_data, y_pred)
metric_results[a_model + f'_{ablation_data}_R_2_before_ablation'] = r2_score(y_data, y_pred)
ablation_results_list = [0] * num_ablate_features
ablation_results_list_r2 = [0] * num_ablate_features
X_temp = X_data.copy()
for i in range(num_ablate_features):
    ablation_X_data = ablation_removal_positive(train_mean, X_temp, local_fi_score_data_rank, local_fi_score_data, i)
    ablation_results_list[i] = mean_squared_error(y_data, ablation_est.predict(ablation_X_data))
    ablation_results_list_r2[i] = r2_score(y_data, ablation_est.predict(ablation_X_data))
    X_temp = ablation_X_data
for i in range(num_ablate_features):
    metric_results[f'{a_model}_{ablation_data}_MSE_after_ablation_{i+1}'] = ablation_results_list[i]
    metric_results[f'{a_model}_{ablation_data}_R_2_after_ablation_{i+1}'] = ablation_results_list_r2[i]

mse_before = metric_results[a_model + f'_{ablation_data}_MSE_before_ablation']
r2_before = metric_results[a_model + f'_{ablation_data}_R_2_before_ablation']

mse_after = [metric_results[f'{a_model}_{ablation_data}_MSE_after_ablation_{i}'] for i in range(1, num_ablate_features)]
r2_after = [metric_results[f'{a_model}_{ablation_data}_R_2_after_ablation_{i}'] for i in range(1, num_ablate_features)]

# Plotting MSE
plt.figure(figsize=(12, 6))
plt.plot(['Before'] + [f'After {i}' for i in range(1, num_ablate_features)], [mse_before] + mse_after, color='tab:red', marker='o')
plt.xlabel('Ablation Step')
plt.ylabel('MSE')
plt.title(f'{a_model} MSE Before and After Ablation')
plt.show()

# Plotting R^2
plt.figure(figsize=(12, 6))
plt.plot(['Before'] + [f'After {i}' for i in range(1, num_ablate_features)], [r2_before] + r2_after, color='tab:blue', marker='o')
plt.xlabel('Ablation Step')
plt.ylabel('R^2')
plt.title(f'{a_model} R^2 Before and After Ablation')
plt.show()

In [ ]:
a_model = "RF_Plus_Regressor"
ablation_est = ablation_models[a_model]
metric_results = {}
local_fi_score_data_rank = np.argsort(-1*np.abs(shap_values_test))#np.argsort(-1*local_feature_importances_train) #np.argsort(-1*shap_values_train) #np.argsort(-1*np.random.rand(X_train.shape[0], 10))
num_ablate_features = X_train.shape[1]
train_mean = np.mean(X_train, axis=0)

y_pred = ablation_est.predict(X_data)
metric_results[a_model + f'_{ablation_data}_MSE_before_ablation'] = mean_squared_error(y_data, y_pred)
metric_results[a_model + f'_{ablation_data}_R_2_before_ablation'] = r2_score(y_data, y_pred)
ablation_results_list = [0] * num_ablate_features
ablation_results_list_r2 = [0] * num_ablate_features
X_temp = X_data.copy()
for i in range(num_ablate_features):
    ablation_X_data = ablation_removal(train_mean, X_temp, local_fi_score_data_rank, i)
    ablation_results_list[i] = mean_squared_error(y_data, ablation_est.predict(ablation_X_data))
    ablation_results_list_r2[i] = r2_score(y_data, ablation_est.predict(ablation_X_data))
    X_temp = ablation_X_data
for i in range(num_ablate_features):
    metric_results[f'{a_model}_{ablation_data}_MSE_after_ablation_{i+1}'] = ablation_results_list[i]
    metric_results[f'{a_model}_{ablation_data}_R_2_after_ablation_{i+1}'] = ablation_results_list_r2[i]

mse_before = metric_results[a_model + f'_{ablation_data}_MSE_before_ablation']
r2_before = metric_results[a_model + f'_{ablation_data}_R_2_before_ablation']

mse_after = [metric_results[f'{a_model}_{ablation_data}_MSE_after_ablation_{i}'] for i in range(1, num_ablate_features)]
r2_after = [metric_results[f'{a_model}_{ablation_data}_R_2_after_ablation_{i}'] for i in range(1, num_ablate_features)]

# Plotting MSE
plt.figure(figsize=(12, 6))
plt.plot(['Before'] + [f'After {i}' for i in range(1, num_ablate_features)], [mse_before] + mse_after, color='tab:red', marker='o')
plt.xlabel('Ablation Step')
plt.ylabel('MSE')
plt.title(f'{a_model} MSE Before and After Ablation')
plt.show()

# Plotting R^2
plt.figure(figsize=(12, 6))
plt.plot(['Before'] + [f'After {i}' for i in range(1, num_ablate_features)], [r2_before] + r2_after, color='tab:blue', marker='o')
plt.xlabel('Ablation Step')
plt.ylabel('R^2')
plt.title(f'{a_model} R^2 Before and After Ablation')
plt.show()

### Check whether sum to hat_y

In [ ]:
assert np.all(rf.predict(X_test)) == np.all(np.sum(shap_values_test, axis=1) + explainer.expected_value)

In [ ]:
explainer.expected_value

In [ ]:
# get localMDI+
rf_plus_mdi = AloRFPlusMDI(rf_plus_base, evaluate_on="all")
local_feature_importances_train, a = rf_plus_mdi.explain(X=X_train, y=y_train)
local_feature_importances_train[0]

In [ ]:
from sklearn.linear_model import Ridge
rf_plus_base = RandomForestPlusRegressor(rf_model=copy.deepcopy(rf), include_raw=False, fit_on="inbag", prediction_model=Ridge(alpha=1e-6))
rf_plus_base.fit(X_train, y_train)
rf_plus_mdi = RFPlusMDI(rf_plus_base, evaluate_on="inbag")
local_feature_importances_train, _ = rf_plus_mdi.explain(X=X_train, y=y_train)
local_feature_importances_train[0]

### LMDI+

In [ ]:
a_model = "Linear"
ablation_est = ablation_models[a_model]
X_data = X_train
y_data = y_train
ablation_data="test"
metric_results = {}
local_fi_score_data =local_feature_importances_train
local_fi_score_data_rank = np.argsort(local_feature_importances_train)
num_ablate_features = X_train.shape[1]
train_mean = np.mean(X_train, axis=0)
y_pred = ablation_est.predict(X_data)
metric_results[a_model + f'_{ablation_data}_MSE_before_ablation'] = mean_squared_error(y_data, y_pred)
metric_results[a_model + f'_{ablation_data}_R_2_before_ablation'] = r2_score(y_data, y_pred)
ablation_results_list = [0] * num_ablate_features
ablation_results_list_r2 = [0] * num_ablate_features
X_temp = X_data.copy()
for i in range(num_ablate_features):
    ablation_X_data = ablation_removal_negative(train_mean, X_temp, local_fi_score_data_rank, local_fi_score_data, i)
    ablation_results_list[i] = mean_squared_error(y_data, ablation_est.predict(ablation_X_data))
    ablation_results_list_r2[i] = r2_score(y_data, ablation_est.predict(ablation_X_data))
    X_temp = ablation_X_data
for i in range(num_ablate_features):
    metric_results[f'{a_model}_{ablation_data}_MSE_after_ablation_{i+1}'] = ablation_results_list[i]
    metric_results[f'{a_model}_{ablation_data}_R_2_after_ablation_{i+1}'] = ablation_results_list_r2[i]

mse_before = metric_results[a_model + f'_{ablation_data}_MSE_before_ablation']
r2_before = metric_results[a_model + f'_{ablation_data}_R_2_before_ablation']

mse_after = [metric_results[f'{a_model}_{ablation_data}_MSE_after_ablation_{i}'] for i in range(1, num_ablate_features)]
r2_after = [metric_results[f'{a_model}_{ablation_data}_R_2_after_ablation_{i}'] for i in range(1, num_ablate_features)]

# Plotting MSE
plt.figure(figsize=(12, 6))
plt.plot(['Before'] + [f'After {i}' for i in range(1, num_ablate_features)], [mse_before] + mse_after, color='tab:red', marker='o')
plt.xlabel('Ablation Step')
plt.ylabel('MSE')
plt.title(f'{a_model} MSE Before and After Ablation')
plt.show()

# Plotting R^2
plt.figure(figsize=(12, 6))
plt.plot(['Before'] + [f'After {i}' for i in range(1, num_ablate_features)], [r2_before] + r2_after, color='tab:blue', marker='o')
plt.xlabel('Ablation Step')
plt.ylabel('R^2')
plt.title(f'{a_model} R^2 Before and After Ablation')
plt.show()

In [ ]:
a_model = "Linear"
ablation_est = ablation_models[a_model]
X_data = X_train
y_data = y_train
ablation_data="test"
metric_results = {}
local_fi_score_data =local_feature_importances_train
local_fi_score_data_rank = np.argsort(-1*local_feature_importances_train)
num_ablate_features = X_train.shape[1]
train_mean = np.mean(X_train, axis=0)
y_pred = ablation_est.predict(X_data)
metric_results[a_model + f'_{ablation_data}_MSE_before_ablation'] = mean_squared_error(y_data, y_pred)
metric_results[a_model + f'_{ablation_data}_R_2_before_ablation'] = r2_score(y_data, y_pred)
ablation_results_list = [0] * num_ablate_features
ablation_results_list_r2 = [0] * num_ablate_features
X_temp = X_data.copy()
for i in range(num_ablate_features):
    ablation_X_data = ablation_removal_positive(train_mean, X_temp, local_fi_score_data_rank, local_fi_score_data, i)
    ablation_results_list[i] = mean_squared_error(y_data, ablation_est.predict(ablation_X_data))
    ablation_results_list_r2[i] = r2_score(y_data, ablation_est.predict(ablation_X_data))
    X_temp = ablation_X_data
for i in range(num_ablate_features):
    metric_results[f'{a_model}_{ablation_data}_MSE_after_ablation_{i+1}'] = ablation_results_list[i]
    metric_results[f'{a_model}_{ablation_data}_R_2_after_ablation_{i+1}'] = ablation_results_list_r2[i]

mse_before = metric_results[a_model + f'_{ablation_data}_MSE_before_ablation']
r2_before = metric_results[a_model + f'_{ablation_data}_R_2_before_ablation']

mse_after = [metric_results[f'{a_model}_{ablation_data}_MSE_after_ablation_{i}'] for i in range(1, num_ablate_features)]
r2_after = [metric_results[f'{a_model}_{ablation_data}_R_2_after_ablation_{i}'] for i in range(1, num_ablate_features)]

# Plotting MSE
plt.figure(figsize=(12, 6))
plt.plot(['Before'] + [f'After {i}' for i in range(1, num_ablate_features)], [mse_before] + mse_after, color='tab:red', marker='o')
plt.xlabel('Ablation Step')
plt.ylabel('MSE')
plt.title(f'{a_model} MSE Before and After Ablation')
plt.show()

# Plotting R^2
plt.figure(figsize=(12, 6))
plt.plot(['Before'] + [f'After {i}' for i in range(1, num_ablate_features)], [r2_before] + r2_after, color='tab:blue', marker='o')
plt.xlabel('Ablation Step')
plt.ylabel('R^2')
plt.title(f'{a_model} R^2 Before and After Ablation')
plt.show()

In [ ]:
a_model = "Linear"
ablation_est = ablation_models[a_model]
X_data = X_train
y_data = y_train
ablation_data="test"
metric_results = {}
local_fi_score_data_rank = np.argsort(-1*np.abs(local_feature_importances_train))#np.argsort(-1*local_feature_importances_train) #np.argsort(-1*shap_values_train) #np.argsort(-1*np.random.rand(X_train.shape[0], 10))
num_ablate_features = X_train.shape[1]
train_mean = np.mean(X_train, axis=0)

y_pred = ablation_est.predict(X_data)
metric_results[a_model + f'_{ablation_data}_MSE_before_ablation'] = mean_squared_error(y_data, y_pred)
metric_results[a_model + f'_{ablation_data}_R_2_before_ablation'] = r2_score(y_data, y_pred)
ablation_results_list = [0] * num_ablate_features
ablation_results_list_r2 = [0] * num_ablate_features
X_temp = X_data.copy()
for i in range(num_ablate_features):
    ablation_X_data = ablation_removal(train_mean, X_temp, local_fi_score_data_rank, i)
    ablation_results_list[i] = mean_squared_error(y_data, ablation_est.predict(ablation_X_data))
    ablation_results_list_r2[i] = r2_score(y_data, ablation_est.predict(ablation_X_data))
    X_temp = ablation_X_data
for i in range(num_ablate_features):
    metric_results[f'{a_model}_{ablation_data}_MSE_after_ablation_{i+1}'] = ablation_results_list[i]
    metric_results[f'{a_model}_{ablation_data}_R_2_after_ablation_{i+1}'] = ablation_results_list_r2[i]

mse_before = metric_results[a_model + f'_{ablation_data}_MSE_before_ablation']
r2_before = metric_results[a_model + f'_{ablation_data}_R_2_before_ablation']

mse_after = [metric_results[f'{a_model}_{ablation_data}_MSE_after_ablation_{i}'] for i in range(1, num_ablate_features)]
r2_after = [metric_results[f'{a_model}_{ablation_data}_R_2_after_ablation_{i}'] for i in range(1, num_ablate_features)]

# Plotting MSE
plt.figure(figsize=(12, 6))
plt.plot(['Before'] + [f'After {i}' for i in range(1, num_ablate_features)], [mse_before] + mse_after, color='tab:red', marker='o')
plt.xlabel('Ablation Step')
plt.ylabel('MSE')
plt.title(f'{a_model} MSE Before and After Ablation')
plt.show()

# Plotting R^2
plt.figure(figsize=(12, 6))
plt.plot(['Before'] + [f'After {i}' for i in range(1, num_ablate_features)], [r2_before] + r2_after, color='tab:blue', marker='o')
plt.xlabel('Ablation Step')
plt.ylabel('R^2')
plt.title(f'{a_model} R^2 Before and After Ablation')
plt.show()

#### Analysis 1

In [ ]:
pd.DataFrame(X_train).corr()

In [ ]:
local_fi_score_data = np.argsort(-1*local_feature_importances_train)#np.argsort(-1*local_feature_importances_train) #np.argsort(-1*shap_values_train) #np.argsort(-1*np.random.rand(X_train.shape[0], 10))

y_pred = ablation_est.predict(X_data)
metric_results[a_model + f'_{ablation_data}_MSE_before_ablation'] = mean_squared_error(y_data, y_pred)
metric_results[a_model + f'_{ablation_data}_R_2_before_ablation'] = r2_score(y_data, y_pred)
imp_vals = copy.deepcopy(local_fi_score_data)
ablation_results_list = [0] * num_ablate_features
ablation_results_list_r2 = [0] * num_ablate_features
X_temp = X_data.copy()
for i in range(num_ablate_features):
    ablation_X_data = ablation_removal_positive(train_mean, X_temp, imp_vals, i)
    ablation_results_list[i] = mean_squared_error(y_data, ablation_est.predict(ablation_X_data))
    ablation_results_list_r2[i] = r2_score(y_data, ablation_est.predict(ablation_X_data))
    X_temp = ablation_X_data
    differences = X_temp != X_train
    differences_per_col = np.sum(differences, axis=0)
    print(differences_per_col)
    lm = LinearRegression()
    lm.fit(X_temp, y_train)
    print(r2_score(y_train, lm.predict(X_temp)))
for i in range(num_ablate_features):
    metric_results[f'{a_model}_{ablation_data}_MSE_after_ablation_{i+1}'] = ablation_results_list[i]
    metric_results[f'{a_model}_{ablation_data}_R_2_after_ablation_{i+1}'] = ablation_results_list_r2[i]

In [ ]:
local_fi_score_data = np.argsort(-1*np.random.rand(X_train.shape[0], 10))#np.argsort(-1*local_feature_importances_train) #np.argsort(-1*shap_values_train) #np.argsort(-1*np.random.rand(X_train.shape[0], 10))

y_pred = ablation_est.predict(X_data)
metric_results[a_model + f'_{ablation_data}_MSE_before_ablation'] = mean_squared_error(y_data, y_pred)
metric_results[a_model + f'_{ablation_data}_R_2_before_ablation'] = r2_score(y_data, y_pred)
imp_vals = copy.deepcopy(local_fi_score_data)
ablation_results_list = [0] * num_ablate_features
ablation_results_list_r2 = [0] * num_ablate_features
X_temp = X_data.copy()
for i in range(num_ablate_features):
    ablation_X_data = ablation_removal_positive(train_mean, X_temp, imp_vals, i)
    ablation_results_list[i] = mean_squared_error(y_data, ablation_est.predict(ablation_X_data))
    ablation_results_list_r2[i] = r2_score(y_data, ablation_est.predict(ablation_X_data))
    X_temp = ablation_X_data
    differences = X_temp != X_train
    differences_per_col = np.sum(differences, axis=0)
    print(differences_per_col)
    lm = LinearRegression()
    lm.fit(X_temp, y_train)
    print(r2_score(y_train, lm.predict(X_temp)))
for i in range(num_ablate_features):
    metric_results[f'{a_model}_{ablation_data}_MSE_after_ablation_{i+1}'] = ablation_results_list[i]
    metric_results[f'{a_model}_{ablation_data}_R_2_after_ablation_{i+1}'] = ablation_results_list_r2[i]

In [ ]:
mse_before = metric_results['Linear_train_MSE_before_ablation']
r2_before = metric_results['Linear_train_R_2_before_ablation']

mse_after = [metric_results[f'Linear_train_MSE_after_ablation_{i}'] for i in range(1, 10)]
r2_after = [metric_results[f'Linear_train_R_2_after_ablation_{i}'] for i in range(1, 10)]

# Plotting MSE
plt.figure(figsize=(12, 6))
plt.plot(['Before'] + [f'After {i}' for i in range(1, 10)], [mse_before] + mse_after, color='tab:red', marker='o')
plt.xlabel('Ablation Step')
plt.ylabel('MSE')
plt.title(f'{a_model} MSE Before and After Ablation')
plt.show()

# Plotting R^2
plt.figure(figsize=(12, 6))
plt.plot(['Before'] + [f'After {i}' for i in range(1, 10)], [r2_before] + r2_after, color='tab:blue', marker='o')
plt.xlabel('Ablation Step')
plt.ylabel('R^2')
plt.title(f'{a_model} R^2 Before and After Ablation')
plt.show()

In [ ]:
local_fi_score_data = np.argsort(-1*shap_values_train)#np.argsort(-1*local_feature_importances_train) #np.argsort(-1*shap_values_train) #np.argsort(-1*np.random.rand(X_train.shape[0], 10))

y_pred = ablation_est.predict(X_data)
metric_results[a_model + f'_{ablation_data}_MSE_before_ablation'] = mean_squared_error(y_data, y_pred)
metric_results[a_model + f'_{ablation_data}_R_2_before_ablation'] = r2_score(y_data, y_pred)
imp_vals = copy.deepcopy(local_fi_score_data)
ablation_results_list = [0] * num_ablate_features
ablation_results_list_r2 = [0] * num_ablate_features
X_temp = X_data.copy()
for i in range(num_ablate_features):
    ablation_X_data = ablation_removal_positive(train_mean, X_temp, imp_vals, i)
    ablation_results_list[i] = mean_squared_error(y_data, ablation_est.predict(ablation_X_data))
    ablation_results_list_r2[i] = r2_score(y_data, ablation_est.predict(ablation_X_data))
    X_temp = ablation_X_data
    differences = X_temp != X_train
    differences_per_col = np.sum(differences, axis=0)
    print(differences_per_col)
    lm = LinearRegression()
    lm.fit(X_temp, y_train)
    print(r2_score(y_train, lm.predict(X_temp)))
for i in range(num_ablate_features):
    metric_results[f'{a_model}_{ablation_data}_MSE_after_ablation_{i+1}'] = ablation_results_list[i]
    metric_results[f'{a_model}_{ablation_data}_R_2_after_ablation_{i+1}'] = ablation_results_list_r2[i]

In [ ]:
mse_before = metric_results['Linear_train_MSE_before_ablation']
r2_before = metric_results['Linear_train_R_2_before_ablation']

mse_after = [metric_results[f'Linear_train_MSE_after_ablation_{i}'] for i in range(1, 10)]
r2_after = [metric_results[f'Linear_train_R_2_after_ablation_{i}'] for i in range(1, 10)]

# Plotting MSE
plt.figure(figsize=(12, 6))
plt.plot(['Before'] + [f'After {i}' for i in range(1, 10)], [mse_before] + mse_after, color='tab:red', marker='o')
plt.xlabel('Ablation Step')
plt.ylabel('MSE')
plt.title(f'{a_model} MSE Before and After Ablation')
plt.show()

# Plotting R^2
plt.figure(figsize=(12, 6))
plt.plot(['Before'] + [f'After {i}' for i in range(1, num_ablate_features)], [r2_before] + r2_after, color='tab:blue', marker='o')
plt.xlabel('Ablation Step')
plt.ylabel('R^2')
plt.title(f'{a_model} R^2 Before and After Ablation')
plt.show()

In [ ]:
y_pred = ablation_est.predict(X_data)
metric_results[a_model + f'_{ablation_data}_MSE_before_ablation'] = mean_squared_error(y_data, y_pred)
metric_results[a_model + f'_{ablation_data}_R_2_before_ablation'] = r2_score(y_data, y_pred)
imp_vals = copy.deepcopy(local_fi_score_data)
ablation_results_list = [0] * num_ablate_features
ablation_results_list_r2 = [0] * num_ablate_features
X_temp = X_data.copy()
var = []
for i in range(num_ablate_features):
    ablation_X_data = ablation_removal_positive(train_mean, X_temp, imp_vals, i)
    ablation_results_list[i] = mean_squared_error(y_data, ablation_est.predict(ablation_X_data))
    ablation_results_list_r2[i] = r2_score(y_data, ablation_est.predict(ablation_X_data))
    X_temp = ablation_X_data
    differences = X_temp != X_train
    differences_per_col = np.sum(differences, axis=0)
    # print(differences_per_col)
    # print(np.array(differences_per_col).var())
    var.append(np.array(differences_per_col).var())

In [ ]:
ablation_results_list

In [ ]:
ablation_results_list_r2

In [ ]:
differences = X_temp != X_train
differences_per_row = np.sum(differences, axis=1)
print(differences_per_row)

In [ ]:
X_train.shape

In [ ]:
differences = X_temp != X_train
differences_per_col = np.sum(differences, axis=0)
print(differences_per_col)
print(np.array(differences_per_col).var())

In [ ]:
train_mean

In [ ]:
ablation_est.coef_

In [ ]:
ablation_est.intercept_

In [ ]:
y_train.mean()

In [ ]:
((y_train - y_train.mean())**2).sum()

In [ ]:
((y_train - ablation_est.predict(X_train))**2).sum()

In [ ]:
((y_train - ablation_est.predict(X_temp))**2).sum()

### Analysis 2

In [ ]:
lm = LinearRegression()
lm.fit(X_temp, y_train)
r2_score(y_train, lm.predict(X_temp))

In [ ]:
lm.coef_

In [ ]:
ablation_est.coef_

In [ ]:
lm.coef_-ablation_est.coef_